In [1]:
# Instalar dependências se necessário (descomente para usar)
# !pip install -r "C:/Users/IA/Desktop/citology pipeline Train/Dataset/pre-processado/3 Classes/requirements.txt"

import sys
from pathlib import Path
import os
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
from IPython.display import display


In [2]:
from typing import Tuple, Dict

DEFAULT_EXTS = ('.jpg', '.jpeg', '.png')

def count_images_in_dir(class_dir: Path, exts=DEFAULT_EXTS, aug_prefix: str = 'aug') -> Tuple[int, int]:
    """Retorna (original_count, augmented_count) para a pasta dada.
    Um arquivo é considerado 'augmented' se seu nome startswith(aug_prefix) (case-insensitive).
    """
    orig = 0
    aug = 0
    prefix_low = aug_prefix.lower()
    for p in class_dir.iterdir():
        if not p.is_file():
            continue
        if p.suffix.lower() not in exts:
            continue
        name_low = p.name.lower()
        if name_low.startswith(prefix_low):
            aug += 1
        else:
            orig += 1
    return orig, aug

def scan_root(root: Path, exts=DEFAULT_EXTS, aug_prefix: str = 'aug') -> Dict[str, Tuple[int, int]]:
    results = {}
    for child in sorted(root.iterdir()):
        if child.is_dir():
            orig, aug = count_images_in_dir(child, exts=exts, aug_prefix=aug_prefix)
            results[child.name] = (orig, aug)
    return results

def make_dataframe(counts: Dict[str, Tuple[int, int]]):
    rows = []
    for cls, (orig, aug) in counts.items():
        rows.append({'class': cls, 'original': orig, 'augmented': aug, 'total': orig + aug})
    df = pd.DataFrame(rows)
    if df.empty:
        return df
    df = df.sort_values(by='total', ascending=False).reset_index(drop=True)
    return df

In [3]:
def plot_counts(df, include_aug: bool = True, out: Path = None, show: bool = True):
    """Plota as contagens usando Plotly."""
    if df.empty:
        print('Nenhuma classe encontrada ou nenhuma imagem presente.')
        return

    labels = df['class'].tolist()
    originals = df['original'].tolist()
    augmented = df['augmented'].tolist()
    totals = df['total'].tolist()
    x = list(range(len(labels)))

    # Construir figura com barras (empilhadas se incluir aumentadas)
    fig = go.Figure()
    if include_aug:
        fig.add_trace(go.Bar(x=x, y=originals, name='original', marker_color='#4C72B0', text=originals, textposition='inside'))
        fig.add_trace(go.Bar(x=x, y=augmented, name='augmented', marker_color='#DD8452', text=augmented, textposition='inside'))
        barmode = 'stack'
    else:
        fig.add_trace(go.Bar(x=x, y=originals, name='original', marker_color='#4C72B0', text=originals, textposition='inside'))
        barmode = 'group'

    fig.update_layout(barmode=barmode, 
                      xaxis=dict(tickmode='array', tickvals=x, ticktext=labels, tickangle=45),
                      yaxis=dict(title='Quantidade de imagens (total)'),
                      title='Distribuição de imagens por classe',
                      legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
                      margin=dict(t=80, b=120))

    # Adicionar linhas pontilhadas horizontais até o eixo Y para cada barra (do eixo y até a posição da barra)
    for xi, tot in zip(x, totals):
        # linha desde a borda esquerda do gráfico (x=-0.5) até a barra (xi) à altura total
        fig.add_shape(type='line', x0=-0.5, x1=xi, y0=tot, y1=tot, xref='x', yref='y', line=dict(color='gray', width=1, dash='dot'))
        # anotar o total no centro vertical da barra empilhada (ou no meio da barra simples)
        ypos = tot / 2 if tot > 0 else 0
        fig.add_annotation(x=xi, y=ypos, text=str(tot), showarrow=False, font=dict(color='white' if ypos > 8 else 'black', size=12))

    # Salvar se solicitado (PNG precisa de kaleido; caso não disponível, salva HTML)
    if out:
        try:
            fig.write_image(str(out), scale=2)
            print(f'Gráfico salvo em: {out}')
        except Exception as e:
            alt = out.with_suffix('.html')
            fig.write_html(str(alt))
            print(f'Não foi possível salvar PNG (kaleido ausente). Salvei HTML em: {alt}. Erro: {e}')

    if show:
        display(fig)

In [4]:
# Parâmetros: ajuste conforme necessário
ROOT = Path('C:/Users/IA/Desktop/citology pipeline Train/Dataset/pre-processado/3 Classes')  # pasta contendo as subpastas de classe
AUG_PREFIX = 'aug'  # prefixo que identifica arquivos aumentados
EXTS = ('.jpg', '.jpeg', '.png')
INCLUDE_AUG = True  # se True, plota stack (original + augmented)
OUT_PNG = ROOT / 'counts.png'  # ou None para não salvar
OUT_CSV = ROOT / 'counts.csv'  # ou None para não salvar

print('ROOT=', ROOT)
print('AUG_PREFIX=', AUG_PREFIX)

ROOT= C:\Users\IA\Desktop\citology pipeline Train\Dataset\pre-processado\3 Classes
AUG_PREFIX= aug


In [5]:
# Executa a contagem e gera o gráfico
counts = scan_root(ROOT, exts=EXTS, aug_prefix=AUG_PREFIX)
df = make_dataframe(counts)

if df.empty:
    print('Nenhuma classe encontrada ou nenhuma imagem presente nas classes em', ROOT)
else:
    display(df)

    if OUT_CSV:
        df.to_csv(OUT_CSV, index=False)
        print(f'CSV salvo em: {OUT_CSV}')

    plot_counts(df, include_aug=INCLUDE_AUG, out=OUT_PNG, show=True)

    # resumo geral
    total_images = df['total'].sum() if not df.empty else 0
    print(f'Imagens totais (soma de todas as classes): {total_images}')

,class,original,augmented,total
0,Normal_tiled,17563,0,17563
1,LISIL_tiled,9080,0,9080
2,HISIL_tiled,3792,5208,9000


CSV salvo em: C:\Users\IA\Desktop\citology pipeline Train\Dataset\pre-processado\3 Classes\counts.csv
Não foi possível salvar PNG (kaleido ausente). Salvei HTML em: C:\Users\IA\Desktop\citology pipeline Train\Dataset\pre-processado\3 Classes\counts.html. Erro: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido

Não foi possível salvar PNG (kaleido ausente). Salvei HTML em: C:\Users\IA\Desktop\citology pipeline Train\Dataset\pre-processado\3 Classes\counts.html. Erro: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido



Imagens totais (soma de todas as classes): 35643


**Notas / próximos passos**

- Ajuste `AUG_PREFIX` se seus arquivos aumentados usarem outro padrão (ex: `aug_`, `aug-`).
- Para contar recursivamente em sub-subpastas, a função `scan_root` pode ser adaptada.
- Exemplos de comandos PowerShell:

```powershell
# Rodar o notebook interativamente (abrir no Jupyter)
jupyter notebook 

# Ou gerar o relatório via script Python (fora do notebook)
python .\count_and_plot.py --root 
```

Se quiser, eu atualizo o `README.md` com instruções e exemplos específicos do notebook.